In [2]:
from pathlib import Path

import matplotlib as mpl

mpl.use("pdf")
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from lowAltitude_classification.gsd_utils import papermode

papermode(plt=plt, size=12)

# width as measured in inkscape
mm = 1/25.4
width0 = 138.6 * mm
width = width0
height = width / 1.618

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = 'GSD_Gaussian_pyramid.csv'  
df = pd.read_csv(file_path)

df['sigma_numeric'] = df['sigma'].str.extract('(\d+)').astype(int)

mapping = { 0: 1.25,
            1: 2.5,
            2: 5,
            4: 10,
            8: 20
}
df['GSD'] = df['sigma_numeric'].map(mapping)
df = df.drop(index=3)
print(df)


     sigma      F1    pAcc  sigma_numeric    GSD
0  sigma_4  0.3460  0.4527              4  10.00
1  sigma_0  0.4317  0.5368              0   1.25
2  sigma_8  0.1301  0.1999              8  20.00
4  sigma_2  0.4234  0.5334              2   5.00
5  sigma_1  0.4351  0.5453              1   2.50


In [5]:
from scipy.interpolate import interp1d

df = df.sort_values(by='sigma_numeric')
x_positions = np.linspace(0, len(df) - 1, len(df))

interp_F1 = interp1d(np.arange(len(df)), df['F1'] * 100, kind='linear')
interp_pAcc = interp1d(np.arange(len(df)), df['pAcc'] * 100, kind='linear')

even_F1 = interp_F1(x_positions)
even_pAcc = interp_pAcc(x_positions)

fig, ax = plt.subplots()
ax.plot(x_positions, even_F1, marker='o', label='$F1$ Score', linestyle='-', linewidth=2,
    color="xkcd:irish green")
ax.plot(x_positions, even_pAcc, marker='o', label='Pixel Accuracy', linestyle=':', linewidth=2,
    color="xkcd:irish green",)

ax.set_xlabel("Standard Deviation of the Gaussian Blur (\(\sigma\))")
ax.set_ylabel("$F1$ score on \\hspace{2.5em} (\%)")
ax.yaxis.set_label_coords(-0.08, 0.5)
ax.text(-0.0795, 0.570, r"$D_{test}^{drone}$", fontsize=12, color="blue", transform=ax.transAxes, ha='right',
        va='center', rotation=90)
ax.legend()
ax.grid(True)
ax.set_xticks(x_positions)
ax.set_xticklabels(df['sigma_numeric'])

fig.tight_layout()

fig.savefig('GSD_plot.png')
fig.savefig('GSD_plot.pdf')